In [ ]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Adjust the path
csv_files = glob.glob("/kaggle/input/modis-8days-1km/*.csv")

data_frames = []

# Load each CSV and append it to the list
for file in csv_files:
    df = pd.read_csv(file)
    df = df.rename(columns={"lat": "Latitude", "lon": "Longitude",
                       "qc_day": "QC_Day", "qc_night": "QC_Night",
                       "value_day": "LST_Day_1km", "value_night": "LST_Night_1km"})
    data_frames.append(df)

# Concatenate all DataFrames in the list to create a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Sort by date
combined_df = combined_df.sort_values(by='system:index').reset_index(drop=True)

# Save the combined DataFrame to a Parquet file
combined_df.to_parquet("modis_8days_1km.parquet", engine="pyarrow", index=False)


In [ ]:
combined_df.info()

In [ ]:
combined_df['Name'].value_counts()

In [ ]:
# Divide by 10
combined_df["Day_view_time"] = combined_df["Day_view_time"] / 10
combined_df["Night_view_time"] = combined_df["Night_view_time"] / 10

In [ ]:
# Divide by 10
df1["Day_view_time"] = df1["Day_view_time"] / 10
df1["Night_view_time"] = df1["Night_view_time"] / 10

In [ ]:
# Extract the actual date part using regex
combined_df['date'] = combined_df['system:index'].str.extract(r'(\d{4}_\d{2}_\d{2})')
combined_df["date"] = pd.to_datetime(combined_df["date"], format='%Y_%m_%d')

df1['date'] = df1['system:index'].str.extract(r'(\d{4}_\d{2}_\d{2})')
df1["date"] = pd.to_datetime(df1["date"], format='%Y_%m_%d')

In [ ]:
# Increment the columns by 1 for summer time months
combined_df.loc[combined_df["date"].dt.month.isin([4, 5, 6, 7, 8, 9, 10]), ["Day_view_time", "Night_view_time"]] += 1
df1.loc[combined_df["date"].dt.month.isin([4, 5, 6, 7, 8, 9, 10]), ["Day_view_time", "Night_view_time"]] += 1

In [ ]:
df_melted = combined_df.melt(value_vars=["Day_view_time", "Night_view_time"], 
                    var_name="Day/Night", value_name="Observation Time (Hour)")
df_melted1 = df1.melt(value_vars=["Day_view_time", "Night_view_time"], 
                    var_name="Day/Night", value_name="Observation Time (Hour)")

In [ ]:
plt.figure(figsize=(12, 8))

# Create the bar plot with automatic binning
sns.histplot(data=df_melted, x="Observation Time (Hour)", hue="Day/Night", bins=np.arange(df_melted["Observation Time (Hour)"].min() - 0.1, df_melted["Observation Time (Hour)"].max() + 0.5, 0.5), multiple="dodge", palette="Set2")

# Customize plot
plt.xlabel("Observation Time (Hour)", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.xticks(np.arange(int(df_melted["Observation Time (Hour)"].min()), int(df_melted["Observation Time (Hour)"].max()) + 1, 1))
plt.title("Bar Plot of Daily Day&Night Observation Time (10km Buffer)", fontsize=20)
plt.savefig("obv_10km.png")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

# Create the bar plot with automatic binning
sns.histplot(data=df_melted1, x="Observation Time (Hour)", hue="Day/Night", bins=np.arange(df_melted1["Observation Time (Hour)"].min() - 0.1, df_melted1["Observation Time (Hour)"].max() + 0.5, 0.5), multiple="dodge", palette="Set2")

# Customize plot
plt.xlabel("Observation Time (Hour)", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.xticks(np.arange(int(df_melted1["Observation Time (Hour)"].min()), int(df_melted1["Observation Time (Hour)"].max()) + 1, 1))
plt.title("Bar Plot of Daily Day&Night Observation Time (no Buffer)", fontsize=20)
plt.savefig("obv_no.png")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

# Create the bar plot with automatic binning
sns.histplot(data=combined_df, x="Night_view_time", bins=np.arange(combined_df["Night_view_time"].min(), combined_df["Night_view_time"].max() + 0.3, 0.3), color="skyblue", discrete=False)

# Customize plot
plt.xlabel("Time Range (Hour)", fontsize=16)
plt.ylabel("Count", fontsize=16)

plt.title("Bar Plot of Daily Night Observation Time (10km Buffer)", fontsize=20)

plt.savefig("time_10km_night.png")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

# Create the bar plot with automatic binning
sns.histplot(data=df1, x="Day_view_time", bins=np.arange(df1["Day_view_time"].min(), df1["Day_view_time"].max() + 0.3, 0.3), color="skyblue", discrete=False)

# Customize plot
plt.xlabel("Time Range (Hour)", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.title("Bar Plot of Daily Day Observation Time (no Buffer)", fontsize=20)

plt.savefig("time_no.png")
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

# Create the bar plot with automatic binning
sns.histplot(data=df1, x="Night_view_time", bins=np.arange(df1["Night_view_time"].min(), df1["Night_view_time"].max() + 0.3, 0.3), color="skyblue", discrete=False)

# Customize plot
plt.xlabel("Time Range (Hour)", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.title("Bar Plot of Daily Night Observation Time (no Buffer)", fontsize=20)

plt.savefig("time_no_night.png")
plt.show()